# 目標：
1. 利用爬蟲抓取台股歷史資料
2. 配合交易方式打造一個回測並可以持續更新的程式
3. 在每日收盤結束後自動更新資料
4. 若有符合買進或賣出的點會透過LINE通知明天應該買入或賣出多少股票

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pygsheets

In [2]:
def lineNotify(stock_id, flag, shares, line_token):
    msg = f"{flag} {stock_id} {shares}股"
    stickerPackageId = 2
    stickerId = 34
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": "Bearer " + line_token
    }
   
    payload = {"message": msg, "stickerPackageId": stickerPackageId, 'stickerId': stickerId}
    r = requests.post(url, headers = headers, params = payload)
    return r.status_code

In [3]:
def goodinfo_crawler(stock_id):
    headers = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"
    }
    req = requests.get(f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE", headers = headers)
    req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, "html.parser")
    date = soup.select("#row0 > td:nth-child(1)")[0].text
    return {
        "foreign_investor":soup.select("#row0 > td:nth-child(8)")[0].text,   #當天外資買賣張數
        "investment_trust":soup.select("#row0 > td:nth-child(13)")[0].text   #當天投信買賣張數
    }, date


In [4]:
def goodinfo_crawler1(stock_id, df):
    headers = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"
    }
    url1 = f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE"
    url2 = f"https://goodinfo.tw/StockInfo/ShowK_Chart.asp?STOCK_ID={stock_id}&CHT_CAT2=DATE"
    req1 = requests.get(url1, headers = headers)
    req1.encoding = 'utf-8'
    soup1 = BeautifulSoup(req1.text, "html.parser")
    req2 = requests.get(url2, headers = headers)
    req2.encoding = 'utf-8'
    soup2 = BeautifulSoup(req2.text, "html.parser")
    data_last_date = datetime.date(datetime.strptime(df.index[-1], "%Y-%m-%d"))
    count = 0
    while True:
        date = datetime.strptime(soup1.select(f"#row{count} > td:nth-child(1)")[0].text, "%y'%m/%d")
        date = datetime.date(date)
        if date == data_last_date:
            break
        else:
            count += 1
    for i in range(count-1, -1, -1):
        data = { 
            "priceOpen":soup2.select(f"#row{i} > td:nth-child(2) > nobr")[0].text,
            "priceHigh":soup2.select(f"#row{i} > td:nth-child(3) > nobr")[0].text,
            "priceLow":soup2.select(f"#row{i} > td:nth-child(4) > nobr")[0].text,
            "priceClose":soup2.select(f"#row{i} > td:nth-child(5) > nobr")[0].text,
            "foreign_investor":soup1.select(f"#row{i} > td:nth-child(8)")[0].text.lstrip("+").replace(",", ""),
            "investment_trust":soup1.select(f"#row{i} > td:nth-child(13)")[0].text.lstrip("+").replace(",", ""), 
        }
        date = soup1.select(f"#row{i} > td:nth-child(1)")[0].text
        date = datetime.date(datetime.strptime(date, "%y'%m/%d"))
        ser = pd.Series(data, name = date)
        df = df.append(ser)
        df = df.astype('float64')
    return df

In [5]:
def fugle_api_crawler(stock_id, fugle_token): 
    payload = {
        "symbolId":f"{stock_id}",
        "apiToken":fugle_token
    }
    req1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote?", params = payload)
    json_data1 = req1.json()['data']['quote']
    req2 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta?", params = payload)
    json_data2 = req2.json()['data']['meta']
    #當天的開高低收
    return {
        'priceOpen':json_data1['priceOpen']['price'],
        'priceHigh':json_data1['priceHigh']['price'],
        'priceLow':json_data1['priceLow']['price'],
        'priceClose':json_data1['trade']['price'],
    }

In [6]:
def goodinfo_crawler_pastdata(stock_id):
    url1 = f"https://goodinfo.tw/StockInfo/ShowBuySaleChart.asp?STOCK_ID={stock_id}&CHT_CAT=DATE"
    url2 = f"https://goodinfo.tw/StockInfo/ShowK_Chart.asp?STOCK_ID={stock_id}&CHT_CAT2=DATE"
    driver1 = webdriver.Chrome()
    driver2 = webdriver.Chrome()
    driver1.implicitly_wait(5)
    driver2.implicitly_wait(5)
    driver1.get(url1)
    driver2.get(url2)
    #抓取歷史法人買賣張數
    s1 = Select(driver1.find_element_by_css_selector('#divBuySaleDetailData > table > tbody > tr > td > table > tbody > tr > td:nth-child(2) > nobr:nth-child(1) > select'))
    #抓取歷史股價
    s2 = Select(driver2.find_element_by_css_selector('#selK_ChartPeriod'))
    s1.select_by_visible_text('一年')
    s2.select_by_visible_text('一年')
    time.sleep(5)
    soup1 = BeautifulSoup(driver1.page_source, 'html.parser')
    soup2 = BeautifulSoup(driver2.page_source, 'html.parser')
    df = pd.DataFrame({})
    i = 0
    while True:
        try:
            data = { 
                "priceOpen":soup2.select(f"#row{i} > td:nth-child(2) > nobr")[0].text,
                "priceHigh":soup2.select(f"#row{i} > td:nth-child(3) > nobr")[0].text,
                "priceLow":soup2.select(f"#row{i} > td:nth-child(4) > nobr")[0].text,
                "priceClose":soup2.select(f"#row{i} > td:nth-child(5) > nobr")[0].text,
                "foreign_investor":soup1.select(f"#row{i} > td:nth-child(8)")[0].text.lstrip("+").replace(",", ""),
                "investment_trust":soup1.select(f"#row{i} > td:nth-child(13)")[0].text.lstrip("+").replace(",", ""), 
            }
            date = soup1.select(f"#row{i} > td:nth-child(1)")[0].text
            date = datetime.strptime(date, "%y'%m/%d")
            ser = pd.Series(data, name = date)
            df = df.append(ser)
            print(stock_id, date)
            i += 1
        except:
            break
    for index, row in df.iterrows():
        if (df.loc[index, "foreign_investor"] == "") or (df.loc[index, "investment_trust"] == ""):
            df = df.drop(index)
    driver1.close()
    driver2.close()
    df = df.sort_index()
    df = df.astype('float64')
    return df


In [7]:
def trading1(df, balance, stock_id, fee, tax, line_token):
    df['buy_or_sell'] = np.zeros(len(df)).reshape((len(df)), 1)    #先將明天買賣股數設為0
    df['shares'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將股數設為0
    df['balance'] = np.zeros(len(df)).reshape((len(df)), 1)  #先將餘額設為0
    df.iloc[0, -1] = balance   #將一開始的餘額設成輸入的值
    df['income'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將損益設為0
    df['profit'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將報酬率設為0
    df['last_close'] = df['priceClose'].shift(1)  #今天收盤價是明天的昨收價
    df['ten_ma'] = df['priceClose'].rolling(10).mean()  #十日平均
    df['IT_flag'] = df['investment_trust'] > 0   #投信是否買超
    df['IT_buy_days'] = df['IT_flag'].rolling(3).sum()   #投信這三天買超的天數
    #當投信連續買超 且 外資今天也買超 且 大於十日平均 就設為買進訊號
    df['buy_flag'] = (df['IT_buy_days'] == 3) & (df['foreign_investor'] > 0) & (df['priceClose'] > df['ten_ma'])
    #當投信不再連續買超 或 收盤價小於十日平均 就設為賣出訊號
    df['sell_flag'] = (df['investment_trust'] < 0) | (df['priceClose'] < df['ten_ma'])   
    count = 0
    win_times = 0 
    before_income = []
    for index, row in df.iterrows():
        #因為第一天沒有昨天的資料所以會出現Error，因此使用try
        try:
            df.loc[index, 'shares'] = yesterday.shares    #先將今天的股數設成和昨天一樣若有買賣再運算
            df.loc[index, 'balance'] = yesterday.balance  #先將今天的餘額設成和昨天一樣若有買賣再運算
            #當昨天的應買賣股數不等於0時，代表今天會執行買賣
            #(附註1：因為三大法人買賣超收盤之後才會知道因此所有動作都只能隔一天才能執行)
            if yesterday.buy_or_sell != 0 :
                df.loc[index, 'buy_or_sell'] = 0    #將應買賣股數變回0
                df.loc[index, 'shares'] = yesterday.shares + yesterday.buy_or_sell   #今日庫存股數為昨日股數加上應買股數
                #今日餘額為昨日餘額加上今天交易股數乘上今天開盤價扣掉手續費和證交稅
                df.loc[index, 'balance'] = yesterday.balance - yesterday.buy_or_sell*df.loc[index, 'priceOpen']*(1- (fee + tax) / 100)  
            
            #當今天買進訊號出現的時候 且 今天的餘額夠買一張股票 明天開盤就買進
            #(附註2：應該使用明天的開盤價，不過明天還沒開盤不會知道開盤價，所以用今天收盤的漲停價來推算餘額夠不夠)
            elif (df.loc[index, 'buy_flag']) and ((df.loc[index, 'balance'] - df.loc[index, 'priceClose'] * 1.1 * 1000) > 0):  
                shares = int(df.loc[index, 'balance'] / (df.loc[index, 'priceClose'] * 1.1 * 1000))   #先估算餘額可以買幾股
                df.loc[index, 'buy_or_sell'] = shares * 1000  #將預估購買的股數填入
                if index == datetime.strftime(datetime.now(), "%Y-%m-%d"):
                    lineNotify(stock_id, "buy", df.loc[index, 'buy_or_sell'], line_token)    #line通知什麼股票應該買多少
            #當今天賣出訊號出現的時候 且 庫存股票股數大於0 明天開盤就賣出
            elif (df.loc[index, 'sell_flag']) and (df.loc[index, 'shares'] > 0):
                df.loc[index, 'buy_or_sell'] = int(df.loc[index, 'shares'] * -1)   #將預估賣出的股數填入
                if index == datetime.strftime(datetime.now(), "%Y-%m-%d"):
                    lineNotify(stock_id, "sell", df.loc[index, 'buy_or_sell'] * -1, line_token)    #line通知什麼股票應該賣多少
    
            df.loc[index, 'income'] = (df.loc[index, 'priceClose'] * df.loc[index, 'shares'] + df.loc[index, 'balance']) - balance
            df.loc[index, 'profit'] = f"{df.loc[index, 'income']/balance *100:.2f}%"
            if df.loc[index, 'buy_or_sell'] > 0:
                before_income.append(df.loc[index, 'income'])
                count += 1    #買賣次數加1
            elif yesterday.buy_or_sell < 0:
                after_income = df.loc[index, 'income']
                before_income = np.array(before_income)
                win_times += len(before_income[before_income < after_income])
                before_income = []
            yesterday = df.loc[index]    #完成資料更新後將其設為下一天的昨天

        except UnboundLocalError:
            yesterday = df.loc[index]
    print(stock_id, win_times, count)
    print(f"{stock_id}：勝率{win_times/count * 100:.2f}% 報酬率{df.iloc[-1, -7]}")
    return df



In [8]:
def backtesting1(start = 0, end = None):
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*float(wks.cell("H2").value)
    tax = float(wks.cell("H3").value)
    balance = int(wks.cell("H4").value)
    line_token = wks.cell("K2").value
    for stock_id in stock_list[start:end]:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data = goodinfo_crawler1(stock_id ,df)
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            data = goodinfo_crawler_pastdata(stock_id)
        result_df = trading1(data, balance, stock_id, fee, tax, line_token)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()
    return result_df

In [9]:
backtesting1()

2337 10 17
2337：勝率58.82% 報酬率19.64%
2884 6 9
2884：勝率66.67% 報酬率3.55%
2882 6 10
2882：勝率60.00% 報酬率6.01%
2313 11 12
2313：勝率91.67% 報酬率72.51%
2330 10 14
2330：勝率71.43% 報酬率16.74%
2368 5 11
2368：勝率45.45% 報酬率0.46%
2456 8 12
2456：勝率66.67% 報酬率32.43%
3037 14 18
3037：勝率77.78% 報酬率43.32%
2458 5 7
2458：勝率71.43% 報酬率9.37%
6213 6 11
6213：勝率54.55% 報酬率15.54%
2886 9 13
2886：勝率69.23% 報酬率15.40%
3481 7 12
3481：勝率58.33% 報酬率12.07%
2409 5 10
2409：勝率50.00% 報酬率12.71%
9933 4 9
9933：勝率44.44% 報酬率12.03%
3189 10 12
3189：勝率83.33% 報酬率33.05%
3227 2 9
3227：勝率22.22% 報酬率8.22%
2379 5 8
2379：勝率62.50% 報酬率11.01%
3044 4 6
3044：勝率66.67% 報酬率6.90%
3324 9 11
3324：勝率81.82% 報酬率59.62%
2327 7 8
2327：勝率87.50% 報酬率37.70%
2449 4 7
2449：勝率57.14% 報酬率16.99%
2448 7 10
2448：勝率70.00% 報酬率83.83%
3042 3 8
3042：勝率37.50% 報酬率6.66%
2421 2 5
2421：勝率40.00% 報酬率6.13%
9938 5 9
9938：勝率55.56% 報酬率14.21%
5264 1 3
5264：勝率33.33% 報酬率-0.38%
3034 6 10
3034：勝率60.00% 報酬率16.79%
8046 8 13
8046：勝率61.54% 報酬率50.89%
5347 4 6
5347：勝率66.67% 報酬率6.70%
6278 8 12
6278：勝率66.67% 報酬率62.0

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk:batchUpdate?fields=replies%2FaddSheet&alt=json returned "Invalid requests[0].addSheet: A sheet with the name "2105" already exists. Please enter another name.">

![](擷取.PNG)

![](擷取1.PNG)

# 小結
### 缺點
1. 原先想要將程式架構在GCP上，並用Google Scheduler每天自動更新資料，但因為爬取的網站需要下拉式選單選取資料時間長度，而我只想到用Selenium方式抓取資料，但無法用Cloud Function上開啟一個模擬瀏覽器
2. 利用Selenium的方式爬蟲速度緩慢，且用for迴圈一列一列抓取速度又更慢，不過有歷史資料的網站不是沒有法人買賣超，就是資料被隱藏或會阻擋爬蟲  

### 解決方案
將Cloud Function中使用到Selenium的程式碼移除，並利用Google Scheduler每天自動更新資料，但當追蹤清單想加入新的股票時需先手動執行程式  
(嘗試使用GCP Computer Engine設定排程執行，但有點不太知道該怎麼處理)

# 以下為還在測試的部分

In [23]:
def trading2(df, balance, stock_id, fee, tax, line_token):
    import pandas as pd
    import numpy as np
    df['buy_or_sell'] = np.zeros(len(df)).reshape((len(df)), 1)    #先將明天買賣股數設為0
    df['shares'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將股數設為0
    df['balance'] = np.zeros(len(df)).reshape((len(df)), 1)  #先將餘額設為0
    df.iloc[0, -1] = balance   #將一開始的餘額設成輸入的值
    df['income'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將損益設為0
    df['profit'] = np.zeros(len(df)).reshape((len(df)), 1)   #先將報酬率設為0
    df['last_close'] = df['priceClose'].shift(1)  #今天收盤價是明天的昨收價
    df['ten_ma'] = df['priceClose'].rolling(10).mean()  #十日平均
    df['FI_flag'] = df['foreign_investor'] > 0   #投信是否買超
    df['FI_buy_days'] = df['FI_flag'].rolling(3).sum()   #投信這三天買超的天數
    #當投信連續買超 且 外資今天也買超 且 大於十日平均 就設為買進訊號
    df['buy_flag'] = (df['FI_buy_days'] == 3) & (df['investment_trust'] > 0) & (df['priceClose'] > df['ten_ma'])
    #當投信不再連續買超 或 收盤價小於十日平均 就設為賣出訊號
    df['sell_flag'] = (df['foreign_investor'] < 0) | (df['priceClose'] < df['ten_ma'])   
    count = 0
    win_times = 0 
    before_income = []
    for index, row in df.iterrows():
        #因為第一天沒有昨天的資料所以會出現Error，因此使用try
        try:
            df.loc[index, 'shares'] = yesterday.shares    #先將今天的股數設成和昨天一樣若有買賣再運算
            df.loc[index, 'balance'] = yesterday.balance  #先將今天的餘額設成和昨天一樣若有買賣再運算
            #當昨天的應買賣股數不等於0時，代表今天會執行買賣
            #(附註1：因為三大法人買賣超收盤之後才會知道因此所有動作都只能隔一天才能執行)
            if yesterday.buy_or_sell != 0 :
                df.loc[index, 'buy_or_sell'] = 0    #將應買賣股數變回0
                df.loc[index, 'shares'] = yesterday.shares + yesterday.buy_or_sell   #今日庫存股數為昨日股數加上應買股數
                #今日餘額為昨日餘額加上今天交易股數乘上今天開盤價扣掉手續費和證交稅
                df.loc[index, 'balance'] = yesterday.balance - yesterday.buy_or_sell*df.loc[index, 'priceOpen']*(1- (fee + tax) / 100)  
               
            #當今天買進訊號出現的時候 且 今天的餘額夠買一張股票 明天開盤就買進
            #(附註2：應該使用明天的開盤價，不過明天還沒開盤不會知道開盤價，所以用今天收盤的漲停價來推算餘額夠不夠)
            elif (df.loc[index, 'buy_flag']) and ((df.loc[index, 'balance'] - df.loc[index, 'priceClose'] * 1.1 * 1000) > 0):  
                shares = int(df.loc[index, 'balance'] / (df.loc[index, 'priceClose'] * 1.1 * 1000))   #先估算餘額可以買幾股
                df.loc[index, 'buy_or_sell'] = shares * 1000  #將預估購買的股數填入
                if index == datetime.date(datetime.now()):
                    lineNotify(stock_id, "buy", df.loc[index, 'buy_or_sell'], line_token)    #line通知什麼股票應該買多少
            #當今天賣出訊號出現的時候 且 庫存股票股數大於0 明天開盤就賣出
            elif (df.loc[index, 'sell_flag']) and (df.loc[index, 'shares'] > 0):
                df.loc[index, 'buy_or_sell'] = int(df.loc[index, 'shares'] * -1)   #將預估賣出的股數填入
                sell_shares = df.loc[index, 'buy_or_sell']
                if index == datetime.date(datetime.now()):
                    lineNotify(stock_id, "sell", df.loc[index, 'buy_or_sell'] * -1, line_token)    #line通知什麼股票應該賣多少
    
            df.loc[index, 'income'] = (df.loc[index, 'priceClose'] * df.loc[index, 'shares'] + df.loc[index, 'balance']) - balance
            df.loc[index, 'profit'] = f"{df.loc[index, 'income']/balance *100:.2f}%"
            if df.loc[index, 'buy_or_sell'] > 0:
                before_income.append(df.loc[index, 'income'])
                count += 1    #買賣次數加1
            elif yesterday.buy_or_sell < 0:
                after_income = df.loc[index, 'income']
                before_income = np.array(before_income)
                win_times += len(before_income[before_income < after_income])
                before_income = []
            yesterday = df.loc[index]    #完成資料更新後將其設為下一天的昨天

        except UnboundLocalError:
            yesterday = df.loc[index]
    print(stock_id, win_times, count)
    print(f"{stock_id}：勝率{win_times/count * 100:.2f}% 報酬率{df.iloc[-1, -7]}")
    return df



In [7]:
def backtesting():
    import pygsheets
    import pandas as pd
    import numpy as np
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*0.6*2
    tax = float(wks.cell("H2").value)
    balance = int(wks.cell("H3").value)
    fugle_token = wks.cell("K1").value
    line_token = wks.cell("K2").value
    for stock_id in stock_list:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data1, date = goodinfo_crawler(str(stock_id))
            data2 = fugle_api_crawler(str(stock_id), fugle_token)
            data = {**data1, **data2}    #合併dict
            ser = pd.Series(data, name = date)
            df.append(ser) 
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            df = goodinfo_crawler_pastdata(stock_id)
            wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        result_df = trading1(df, balance, stock_id, fee, tax, line_token)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()

In [ ]:
def backtesting2():
    import pygsheets
    import pandas as pd
    import numpy as np
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*0.6*2
    tax = float(wks.cell("H2").value)
    balance = int(wks.cell("H3").value)
    fugle_token = wks.cell("K1").value
    line_token = wks.cell("K2").value
    for stock_id in stock_list:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data = goodinfo_crawler1(stock_id ,df)
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            data = goodinfo_crawler_pastdata(stock_id)
        result_df = trading2(data, balance, stock_id, fee, tax, line_token)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()
    return result_df

In [16]:
backtesting2()

AttributeError: 'NoneType' object has no attribute 'tag_name'

In [171]:
def BBand(stock_id, df):
    import xlwings as xw
    import pandas as pd
    import numpy as np
    df['mean'] = df['收盤價'].rolling(20).mean()
    df['max'] = df['mean'] + 2 * df['收盤價'].rolling(20).std()
    df['min'] = df['mean'] - 2 * df['收盤價'].rolling(20).std()
    df = df.set_index('日期')
    df['mean'] = df['mean'].shift(1)
    df['max'] = df['max'].shift(1)
    df['min'] = df['min'].shift(1)
    
    hold_flag = False
    long_flag = False
    short_flag = False
    shares = 1000
    balance = 0
    for index, row in df.iterrows():
        if hold_flag == False:
            if row['收盤價'] >= row['max']:
                long_flag = True
                df.loc[index, '交易訊號'] = 'Long'
                balance = balance - row['收盤價'] * shares
                df.loc[index, '損益'] = balance
                hold_flag = True
            if row['收盤價'] < row['min']:
                short_flag = True
                df.loc[index, '交易訊號'] = 'Short'
                balance = balance + row['收盤價'] * shares
                df.loc[index, '損益'] = balance    
                hold_flag = True
            else:
                df.loc[index, "交易訊號"] = "---"
                df.loc[index, "損益"] = balance
        if hold_flag == True:
            if (long_flag) and (row["收盤價"] <= row["max"]):
                df.loc[index, '交易訊號'] = 'Offset'
                df.loc[index, '損益'] = df.loc[index, '損益'] + row["收盤價"] * shares
                hold_flag = False
                long_flag = False
            if (short_flag) and (row["收盤價"] >= row["min"]):
                df.loc[index, '交易訊號'] = 'Offset'
                df.loc[index, '損益'] = df.loc[index, '損益'] - row["收盤價"] * shares
                hold_flag = False
                short_flag = False
        else:
            df3.loc[index, "交易訊號"] = "---"
            df3.loc[index, "損益"] = balance
        return xw.view(df3)

In [174]:
def backtesting3():
    import pygsheets
    import pandas as pd
    import numpy as np
    gc = pygsheets.authorize(service_account_file=r"C:\Users\j3192\Desktop\Jupyter Notebook\台大程式課程\ntu_pyxl_lesson10\pyxl-271804-be731475b523.json")
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1SJ9YRMvoChR8F2nb1H8rxjR5vClyINSIjlcrLsSw6Nk/edit#gid=0")
    wks = wb.worksheet_by_title("追蹤清單")
    stock_list = wks.get_col(1, include_tailing_empty=False)
    fee = float(wks.cell("H1").value)*0.6*2
    tax = float(wks.cell("H2").value)
    balance = int(wks.cell("H3").value)
    fugle_token = wks.cell("K1").value
    line_token = wks.cell("K2").value
    for stock_id in stock_list:
        try:
            wks = wb.worksheet_by_title(str(stock_id))
            cols = len(wks.get_col(1, include_tailing_empty = False))
            df = wks.get_as_df(index_column = 1, end = (cols, 7))
            data = goodinfo_crawler1(stock_id ,df)
        except: 
            wks = wb.add_worksheet(str(stock_id), rows=100, cols=26, src_tuple=None, index=None)
            data = goodinfo_crawler_pastdata(stock_id)
        result_df = BBand(stock_id, data)
        wks.set_dataframe(result_df.loc[:, "foreign_investor":"profit"], 'A1', copy_index=True)
        wks.refresh()
    return result_df

In [175]:
backtesting3()

KeyError: '收盤價'

![](r"C:\Users\j3192\Desktop\擷取.PNG")